In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

## Building Retrieval Models easily with Merlin Models

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are **Matrix Factorization**, which learns low-rank user and item embeddings, and the **Two-Tower architecture**, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output. Such models can be efficiently served by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine and during inference scoring user embeddings over all indexed item embeddings within the engine.In this notebook we will build a Two-Tower architecture for item retrieval.



### Dataset

In this notebook, we are building a Two-Tower model for Item Retrieval task using the Ali-CCP: Alibaba Click and Conversion Prediction dataset. To download the training and test datasets visit Ali-CCP: Alibaba Click and Conversion Prediction at [tianchi.aliyun.com](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1). We have curated the raw dataset via using this [script]() and generated the parquet files that we will use for this example.

### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models
- Perform prediction for a given user query

### Feature Engineering with NVTabular

In [2]:
import os
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"
import cudf
import glob
import gc

import nvtabular as nvt
from nvtabular.ops import *
from example_utils import workflow_fit_transform

from merlin.schema.tags import Tags
from merlin.schema import Schema

import merlin.models.tf as mm
import merlin.models.tf.dataset as tf_dataloader

from merlin.io.dataset import Dataset
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata
from merlin.models.tf.blocks.core.aggregation import CosineSimilarity

import tensorflow as tf

2022-03-22 20:55:13.766145: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-22 20:55:15.178604: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-03-22 20:55:15.178737: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


First, we define our input and output paths.

In [3]:
train_path = '/workspace/data/train/*.parquet'
test_path = '/workspace/data/test/*.parquet'
output_path = '/workspace/retrieval/processed/'

ETL Workflow

We select only positive interaction rows therefore we remove rows where `click==0` from the dataset with `Filter()` op.

In [4]:
user_id = ["user_id"] >> Categorify() >> AddMetadata(tags=[Tags.USER_ID, Tags.USER])
item_id = ["item_id"] >> Categorify() >> AddMetadata(tags=[Tags.ITEM_ID, Tags.ITEM]) 

item_features = ["item_category", "item_shop", "item_brand"] \
     >> AddMetadata(tags=[Tags.ITEM]) >> nvt.ops.Categorify()

user_features = ['user_shops', 'user_profile', 'user_group', 
       'user_gender', 'user_age', 'user_consumption_2', 'user_is_occupied',
       'user_geography', 'user_intentions', 'user_brands', 'user_categories'] \
    >> AddMetadata(tags=[Tags.USER]) >> nvt.ops.Categorify()

inputs = user_id + item_id + item_features + user_features + ['click'] 

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

workflow_fit_transform(outputs, train_path, test_path, output_path)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


## Building Two-Tower Model

We will use Two-Tower Model for Item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. It is Thus, these systems are often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). You can read more about two stage Recommender Systems [here](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45530.pdf). In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

We use the `schema` object to define our model.

In [5]:
schema = TensorflowMetadata.from_proto_text_file('/workspace/retrieval/processed/train/').to_merlin_schema()

In [6]:
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

We expect the label names to be empty.

In [7]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Negative sampling
Many datasets for recommender systems contains implicit feedback, with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones.
In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [8]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),        
    loss="categorical_crossentropy",  
    samplers=[mm.InBatchSampler()],
    embedding_options = mm.EmbeddingOptions(infer_embedding_sizes=True),
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)]
)

Let's explain the parameters in the TwoTowerModel():
- no_activation_last_layer: when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- infer_embedding_sizes: when set True, automatically defines the embedding dimension from the feature cardinality in the schema

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [9]:
batch_size = 4096

train = Dataset(os.path.join(output_path + '/train/*.parquet'), part_size="500MB")
valid = Dataset(os.path.join(output_path + '/test/*.parquet'), part_size="500MB")

In [10]:
model.set_retrieval_candidates_for_evaluation(train)

opt = tf.keras.optimizers.Adagrad(learning_rate=0.003)
model.compile(optimizer=opt, run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=batch_size, epochs=3)

2022-03-22 20:56:02.012540: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


383/384 [============================>.] - ETA: 0s - recall_at_10: 0.0025 - ndcg_10: 0.0012 - loss: 8.3228 - regularization_loss: 0.0000e+00 - total_loss: 8.3228

INFO:tensorflow:Assets written to: /tmp/tmpubcby9_u/assets


...

INFO:tensorflow:Assets written to: /tmp/tmpijams9ux/assets


384/384 [==============================] - 30s 78ms/step - recall_at_10: 0.0025 - ndcg_10: 0.0012 - loss: 8.3190 - regularization_loss: 0.0000e+00 - total_loss: 8.3190 - val_recall_at_10: 0.0025 - val_ndcg_10: 0.0011 - val_loss: 7.4219 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4219
Epoch 3/3
384/384 [==============================] - ETA: 0s - recall_at_10: 0.0026 - ndcg_10: 0.0012 - loss: 8.3198 - regularization_loss: 0.0000e+00 - total_loss: 8.3198

INFO:tensorflow:Assets written to: /tmp/tmpg34wc9x8/assets


INFO:tensorflow:Assets written to: /tmp/tmpg34wc9x8/assets


384/384 [==============================] - 31s 79ms/step - recall_at_10: 0.0026 - ndcg_10: 0.0012 - loss: 8.3184 - regularization_loss: 0.0000e+00 - total_loss: 8.3184 - val_recall_at_10: 0.0025 - val_ndcg_10: 0.0012 - val_loss: 7.4213 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4213


In batch sampling is prone to popularity-bias. since different users might have interacted same items. # in the fit method we are only using the negative sampling in each batch and the final score is avg over all the batches. 
for each batch we aer comparing only the items in the given batch. in the `.evaluate()` we are considering entire item catalog for each positive item. generate the scores for all items and check the position of the positive item in the list of scores.

In [11]:
#model.evaluate(valid, return_dict=True, batch_size=1024)

## Exporting Retrieval Models

#### Save user (query) tower

In [12]:
query_tower = model.retrieval_block.query_block()
query_tower.save('query_tower')

INFO:tensorflow:Assets written to: query_tower/assets


INFO:tensorflow:Assets written to: query_tower/assets


#### Extract and save user features

In [25]:
from merlin.models.utils.dataset import unique_rows_by_features
user_features = unique_rows_by_features(Dataset('/workspace/retrieval/processed/train/*.parquet'), Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [26]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,0,0,1,4,1,3,2,1,0,0,0,0
1,1,109,0,0,0,0,0,0,0,69,131,9
2,2,301,1,1,1,1,1,1,2,57,4709,57
3,3,1876,23,7,2,3,1,1,1,5,63,3
4,4,534,1,2,1,2,1,1,0,40,22,108


In [27]:
user_features.shape

(214994, 12)

In [28]:
#user_features.to_parquet('user_features.parquet')

#### Extract and save item features

In [29]:
# train path should be the path for the processed parquet files.
item_features = unique_rows_by_features(Dataset('/workspace/retrieval/processed/train/*.parquet'), Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


In [30]:
item_features.head()

,item_id,item_category,item_shop,item_brand
0,0,0,0,0
1,1,441,432,474
2,2,193,1159,125
3,3,3,1463,872
4,4,282,2479,555


In [31]:
#item_features.to_parquet('item_features.parquet')

#### Extract and save item embeddings

In [32]:
item_embs = model.item_embeddings(Dataset(item_features, schema=schema), batch_size=1024)
item_embs_df = item_embs.compute(scheduler="synchronous")

INFO:tensorflow:Assets written to: /tmp/tmp1nx5vchy/assets


INFO:tensorflow:Assets written to: /tmp/tmp1nx5vchy/assets


In [33]:
item_embs_df

,item_id,item_category,item_shop,item_brand,0,1,2,3,4,5,...,54,55,56,57,58,59,60,61,62,63
0,0,0,0,0,-0.035471,-0.092948,-0.162513,0.051653,-0.081698,-0.011199,...,-0.082671,0.149540,-0.037409,-0.029979,0.330735,-0.138205,0.103216,0.158921,-0.151992,-0.119814
1,1,441,432,474,0.293629,-0.004668,-0.100611,-0.252162,-0.219130,0.028827,...,-0.211181,0.040126,0.022910,-0.144738,0.313601,-0.093773,-0.076630,0.008872,0.079040,-0.165187
2,2,193,1159,125,0.126375,-0.113511,0.035486,-0.191057,-0.100485,-0.069106,...,0.182612,0.034946,0.131176,0.016516,0.140971,-0.142058,-0.222311,0.054083,-0.269620,-0.145560
3,3,3,1463,872,0.039960,-0.129235,0.095869,0.000588,-0.065913,-0.011629,...,-0.139102,0.038123,0.023675,-0.073021,-0.023763,-0.251495,-0.103316,0.156044,-0.221559,-0.405354
4,4,282,2479,555,0.017206,-0.043477,0.049604,0.150207,-0.111438,0.136317,...,-0.072897,0.046393,0.079116,0.099039,-0.023284,0.019531,0.059016,-0.038120,-0.199234,-0.313679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568842,3078159,1764,365864,121053,0.251918,0.001146,-0.009754,-0.133443,-0.280002,-0.120191,...,0.085881,0.082907,0.088727,0.095871,0.002160,-0.010803,-0.087407,0.029755,-0.075418,-0.291222
568843,3078210,377,97495,116656,-0.073524,-0.110021,-0.028398,-0.097701,-0.238075,-0.169089,...,0.055080,-0.002275,-0.071681,0.276715,0.271313,-0.186708,-0.095671,0.106745,-0.037785,-0.314436
568844,3078270,3817,577371,80796,0.141514,-0.178248,-0.072934,0.010046,-0.127190,-0.175243,...,0.032672,-0.062497,-0.175352,0.117554,0.094920,-0.074599,-0.038968,0.340473,-0.296917,-0.268433
568845,3078283,551,201089,85172,0.175509,0.050898,0.066930,0.012926,-0.028945,-0.044987,...,-0.031746,-0.000695,0.060886,0.178592,0.005817,-0.079405,-0.299065,-0.036055,-0.201436,-0.027930


In [34]:
item_embeddings = item_embs_df.iloc[:, 4:]

In [35]:
item_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.035471,-0.092948,-0.162513,0.051653,-0.081698,-0.011199,0.060544,-0.004139,-0.064105,-0.154147,...,-0.082671,0.149540,-0.037409,-0.029979,0.330735,-0.138205,0.103216,0.158921,-0.151992,-0.119814
1,0.293629,-0.004668,-0.100611,-0.252162,-0.219130,0.028827,0.022169,-0.066528,0.090813,-0.058673,...,-0.211181,0.040126,0.022910,-0.144738,0.313601,-0.093773,-0.076630,0.008872,0.079040,-0.165187
2,0.126375,-0.113511,0.035486,-0.191057,-0.100485,-0.069106,-0.056850,-0.085665,-0.136583,-0.082392,...,0.182612,0.034946,0.131176,0.016516,0.140971,-0.142058,-0.222311,0.054083,-0.269620,-0.145560
3,0.039960,-0.129235,0.095869,0.000588,-0.065913,-0.011629,-0.000011,-0.207328,-0.051968,0.056846,...,-0.139102,0.038123,0.023675,-0.073021,-0.023763,-0.251495,-0.103316,0.156044,-0.221559,-0.405354
4,0.017206,-0.043477,0.049604,0.150207,-0.111438,0.136317,-0.076041,-0.038415,0.020472,-0.053883,...,-0.072897,0.046393,0.079116,0.099039,-0.023284,0.019531,0.059016,-0.038120,-0.199234,-0.313679


In [24]:
#item_embeddings.to_parquet('item_embeddings')

In [39]:
train = cudf.concat([cudf.read_parquet(x) for x in glob.glob(train_path)])
train.head()

,item_intention,position,item_id,item_category,item_shop,user_item_categories,user_categories,user_intentions,user_id,user_brands,...,user_item_brands,user_item_intentions,user_gender,user_age,user_consumption_1,user_consumption_2,user_is_occupied,user_geography,user_group,user_profile
0,9069229,9351665,5236823,8315701,8927579,9354021,450937,3942527,354229,3488451,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,9036102,9351665,5369565,8313377,8794331,9351963,450937,3942527,354229,3488451,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,9025172,9351665,5643280,8317268,8722017,<NA>,450937,3942527,354229,3488451,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,9112773,9351665,5730129,8313131,8821712,9351731,450937,3942527,354229,3488451,...,9886929,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,9031696,9351665,5732064,8313213,8695141,9351810,450937,3942527,354229,3488451,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [50]:
train['user_id'].isnull().sum()

90671

In [42]:
tr = train[train['click']==1]

In [52]:
tr['user_id'].isnull().sum()

2501